In [12]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np


In [13]:
# Read CSV
df = pd.read_csv("stock_data.csv")
df['Sentiment'].replace(-1, 0, inplace=True) # change 0's in sentiment to 1's
sentiment = df.Sentiment
text = df.Text

In [14]:
# Tokenize words
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 1000
tokenizer = Tokenizer(num_words = vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(text)
import pickle
f = open('lstm_tokenizer.pickle', 'wb')
pickle.dump(tokenizer, f)
f.close()
word_index = tokenizer.word_index # dictionary of words and values
sequences = tokenizer.texts_to_sequences(text)

In [15]:
# Pad vectors that represent each sentence
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(sequences)


In [16]:
# Create Model and Vectorize Words
sentiment_model = Sequential()
num_features = 128
sentiment_model.add(tf.keras.layers.Embedding(vocab_size, num_features, input_length = len(padded_sequences[0])))
sentiment_model.add(tf.keras.layers.LSTM(num_features))
sentiment_model.add(tf.keras.layers.Dropout(0.3))
sentiment_model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
sentiment_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
sentiment_model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 34, 128)           128000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 259,713
Trainable params: 259,713
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(padded_sequences), np.array(sentiment), test_size=0.2)

In [18]:
# Train Model
sentiment_model.fit(X_train, y_train, epochs = 5, validation_data = (X_test, y_test), batch_size = 64)

Epoch 1/5
73/73 [==============================] - 5s 57ms/step - loss: 0.6180 - accuracy: 0.6641 - val_loss: 0.5169 - val_accuracy: 0.7299
Epoch 2/5
73/73 [==============================] - 3s 39ms/step - loss: 0.4642 - accuracy: 0.7761 - val_loss: 0.4605 - val_accuracy: 0.7886
Epoch 3/5
73/73 [==============================] - 2s 31ms/step - loss: 0.3933 - accuracy: 0.8204 - val_loss: 0.4715 - val_accuracy: 0.7834
Epoch 4/5
73/73 [==============================] - 2s 32ms/step - loss: 0.3697 - accuracy: 0.8351 - val_loss: 0.4924 - val_accuracy: 0.7826
Epoch 5/5
73/73 [==============================] - 2s 31ms/step - loss: 0.3393 - accuracy: 0.8545 - val_loss: 0.4924 - val_accuracy: 0.7852


In [19]:
# test results
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix
y_prob = sentiment_model.predict(X_test)

y_pred = []
for i in y_prob:
    y_pred.append(round(i[0]))
print(accuracy_score(y_pred, y_test))
confusion_matrix(y_test, y_pred)

37/37 [==============================] - 1s 8ms/step
0.7851596203623814


array([[268, 149],
       [100, 642]])

In [20]:
import pickle
f = open('lstm_classifier.pickle', 'wb')
pickle.dump(sentiment_model, f)
f.close()

INFO:tensorflow:Assets written to: ram://b0d7266b-3db9-4db4-90cd-8947721bbe91/assets


INFO:tensorflow:Assets written to: ram://b0d7266b-3db9-4db4-90cd-8947721bbe91/assets


In [28]:
f = open("lstm_classifier.pickle",'rb')
classifier = pickle.load(f)
f.close()
f = open("lstm_tokenizer.pickle", 'rb')
tknizer = pickle.load(f)
f.close()

test = ['the stocks look terrible', 'the stock market looks like it is going up today']
test = tokenizer.texts_to_sequences(test)
test = np.array(pad_sequences(test, maxlen=34))
print(classifier.predict(test))

1/1 [==============================] - 0s 253ms/step
[[0.83045447]
 [0.9631648 ]]
